In [0]:
'''
* m_t_d_xle_policy.py
*
* <MappingDescription <<<----------------------------------------------------------------  Developer: Cognizant  Date: 12/27/2012  Purpose: This mapping is used to populate the dimension table of policy with type 2 changes.  ---------------------------------------------------------------------------------------->>>
* <MappingDescription <<<-----------------------------------------------------------------  Developer: Cognizant  Date: 22/11/2013  Purpose: This mapping is used to populate the new hash trn table>>>
* <MappingDescription <<<==================================================Date Created:  1/11/2013   Developer:  Cognizant  Purpose : This Mapping is to call stored procedures to claeanup the data in Fact tables and Previous hash tables   ==================================================  >>>
* <MappingDescription <<<-----------------------------------------------------------------  Developer: Cognizant  Date: 22/11/2013  Purpose: This mapping is used to populate the previous hash trn table  ---------------------------------------------------------------------------------------->>>
* <MappingDescription <<<==================================================Date Created:  12/24/2012  Developer: X045436 - Cognizant  Purpose : This Mapping is to Insert/Update batch.t_batch  table  ==================================================  >>>
* <MappingDescription <<<==================================================Date Created:  12/24/2012  Developer:  Cognizant  Purpose : This Mapping is to call stored procedure 'spu_trunc_partition_tables'  which will  delete partition data from the new  hash or current hash or  int tables based on the parameters  ==================================================>>>
* <MappingDescription <<<-- =============================================  -- Author:       Cognizant  -- Create date: 8-Feb-2013  -- Description:  This Shared mapping is used to get the record counts and sum of standard measure amounts from Source and load them to BalanceControlData table  -- =============================================>>>
* <MappingDescription <<<==================================================Date Created:  03/04/2013  Developer: X045436 - Cognizant  Purpose : This Mapping is to Insert/Update batch.t_batch  table based on balanced_ind of audit.t_balance table  ==================================================  >>>
* <MappingDescription <<<-- =============================================  -- Author:       Cognizant  -- Create date: 16-Apr-2013  -- Description:  This Shared mapping is used to get the record counts and sum of standard measure amounts from Source and load them to BalanceControlData table  -- =============================================>>>
* @date    Aug 27, 2021
*
* @author  IDW [idwBuddy@cognizant.com]
*
* @version 1.0
*
* @par Remarks
*
* This code is generated by Cognizant Intelligent Data Works (IDW) Data Engineering Foundry platform.
* Please review the business rules before productionize. 
'''

import pyspark
from pyspark.sql import SparkSession
import datetime
import pandas as pd
from pyspark.sql.functions import * 
from pyspark.sql import DataFrame
from functools import reduce 
from pyspark.sql.types import * 
from pyspark.sql import Window
from typing import Iterable
from itertools import chain
import datetime
import json


#Spark Session initialization
spark = SparkSession.builder.enableHiveSupport().getOrCreate()


In [0]:
%run ../shared/m_load_variables


In [0]:
mapVariable_TARGET_TABLE_NAME_TEST = "t_d_xle_policy_testing" #for testing
t_previous_hashes_trn_Demo="t_previous_hashes_trn_Demo" #override for testing

In [0]:
#RESET TEST DATA
#df_copy=spark.sql("Select * from " + conn_xl_edw_dw_xle_r32_FilePath + "." + mapVariable_TARGET_TABLE_NAME)
#df_copy.write.format("delta").mode('overwrite').option("overwriteSchema",True).save(conn_Target_FilePath + conn_xl_edw_dw_xle_r32_FilePath + "/" + mapVariable_TARGET_TABLE_NAME_TEST)

In [0]:
#Reading Data From Source
#needs to be current_timestamp instead of current_dt 
query ="""
SELECT   
  current_timestamp as current_dt,
  dummy as dummy 
FROM ( SELECT   
        current_date as current_dt,   
        'X' as dummy) t  
"""

t_dw_etl_batch_process_history = spark.sql(query)  #Remove monotonically increasing ID



In [0]:
#reformat and replace schema names with db names.  
#values in SQL query were hard-coded when they should have been variables
#would be nice if conversion can keep SQL formatting

#Reading Data From Source

query ="""
SELECT 
  plcy_src_sys_cd, 
  plcy_textnk, 
  plcy_hashnk, 
  plcy_hashfull, 
  plcy_nbr, 
  plcy_eff_dt, 
  plcy_expr_dt, 
  plcy_binder_mga_ind, 
  plcy_ledger_ri_type_cd, 
  plcy_ledger_ri_type_descr, 
  plcy_days_to_cancel, 
  plcy_dir_assm_ind, 
  plcy_dir_assm_ind_descr, 
  plcy_earn_pattern_method_ind, 
  plcy_old_new_econ_ind, 
  plcy_lta_eff_dt, 
  plcy_lta_expr_dt, 
  plcy_lta_ind, 
  plcy_lead_follow_ind, 
  plcy_new_rnwl_ind, 
  plcy_new_rnwl_ind_descr, 
  plcy_primary_excess_ind, 
  plcy_primary_excess_ind_descr, 
  plcy_prog_ind, 
  plcy_prog_ind_descr, 
  plcy_uw_year, 
  plcy_written_dt, 
  plcy_share_of_whole_pct, 
  prior_plcy_nbr, 
  plcy_prog_master_ind, 
  plcy_prog_master_layer_ind, 
  plcy_orig_eff_dt, 
  plcy_orig_expr_dt, 
  plcy_runoff_ind, 
  plcy_cancelled_dt, 
  plcy_binder_mga_cancel_notice_pd, 
  plcy_market_class_cd, 
  plcy_market_class_descr, 
  plcy_mgmt_triangle_cd, 
  plcy_mgmt_triangle_descr, 
  plcy_risk_class_cd, 
  plcy_risk_class_descr, 
  plcy_syndicate_cd, 
  plcy_syndicate_nme, 
  plcy_syndicate_ssr_cd, 
  plcy_syndicate_ssr_descr,
  plcy_title,
  plcy_risk_band_cd,
  plcy_risk_band_descr,
  s2_plcy_status_cd,
  s2_plcy_status_descr,
  s2_plcy_status_dt,
  plcy_old_new_econ_ind_descr,
  plcy_upr_transition_dt,
  plcy_ny_free_trd_zn_class_cd,
  plcy_ny_free_trd_zn_class_descr,
  plcy_captive_ind,
  plcy_captive_ind_descr,
  plcy_multi_year_ind,
  plcy_ny_free_trd_zn_stat_cd,
  plcy_ny_free_trd_zn_stat_descr,
  plcy_risk_purch_grp_ind,
  plcy_lgl_enty_consolidation_ind,
  legacy_prior_plcy_nbr,
  plcy_facilities_ind,
  frame_underwriting_ref_identifier,
  plcy_xl_risk_ind,
  plcy_axa_ind,
  trans_fin_type_fronting_ind,
  plcy_revenue_synergy_cd,
  plcy_revenue_synergy_descr,
  plcy_revenue_synergy_ind,
  plcy_eea_ind,
  plcy_eea_ind_dt,
  PLcy_ilu_ind,
  plcy_integrated_solution_ind
from 
  """ + conn_xl_edw_interface_xle_r32_FilePath + "." + mapVariable_TARGET_TABLE_NAME + """
where 
  plcy_src_sys_cd='""" +mapVariable_SRC_SYS_CD + """'
union all
SELECT 
  null as plcy_src_sys_cd, 
  null as plcy_textnk, 
  hashnk as plcy_hashnk, 
  hashfull as plcy_hashfull,
  null as plcy_nbr, 
  null as plcy_eff_dt, 
  null as plcy_expr_dt,
  null as plcy_binder_mga_ind, 
  null as plcy_ledger_ri_type_cd, 
  null as plcy_ledger_ri_type_descr, 
  null as plcy_days_to_cancel, 
  null as plcy_dir_assm_ind, 
  null as plcy_dir_assm_ind_descr, 
  null as plcy_earn_pattern_method_ind, 
  null as plcy_old_new_econ_ind, 
  null as plcy_lta_eff_dt, 
  null as plcy_lta_expr_dt, 
  null as plcy_lta_ind, 
  null as plcy_lead_follow_ind, 
  null as plcy_new_rnwl_ind, 
  null as plcy_new_rnwl_ind_descr, 
  null as plcy_primary_excess_ind, 
  null as plcy_primary_excess_ind_descr, 
  null as plcy_prog_ind, 
  null as plcy_prog_ind_descr,
  null as plcy_uw_year, 
  null as plcy_written_dt, 
  null as plcy_share_of_whole_pct,
  null as prior_plcy_nbr, 
  null as plcy_prog_master_ind, 
  null as plcy_prog_master_layer_ind, 
  null as plcy_orig_eff_dt, 
  null as plcy_orig_expr_dt, 
  null as plcy_runoff_ind, 
  null as plcy_cancelled_dt, 
  null as plcy_binder_mga_cancel_notice_pd, null as plcy_market_class_cd, null as plcy_market_class_descr, 
  null as plcy_mgmt_triangle_cd, null as plcy_mgmt_triangle_descr, null as plcy_risk_class_cd,
  null as plcy_risk_class_descr, null as plcy_syndicate_cd, null as plcy_syndicate_nme, 
  null as plcy_syndicate_ssr_cd, null as plcy_syndicate_ssr_descr,
  null as plcy_title,
  null as plcy_risk_band_cd,
  null as plcy_risk_band_descr,
  null as s2_plcy_status_cd,
  null as s2_plcy_status_descr,
  null as s2_plcy_status_dt,
  null as plcy_old_new_econ_ind_descr,
  null as plcy_upr_transition_dt,
  null as plcy_ny_free_trd_zn_class_cd,
  null as plcy_ny_free_trd_zn_class_descr,
  null as plcy_captive_ind, 
  null as plcy_captive_ind_descr, 
  null as plcy_multi_year_ind,
  null as plcy_ny_free_trd_zn_stat_cd,
  null as plcy_ny_free_trd_zn_stat_descr,
  null as plcy_risk_purch_grp_ind,
  null as plcy_lgl_enty_consolidation_ind,
  null as legacy_prior_plcy_nbr,
  null as plcy_facilities_ind,
  null as frame_underwriting_ref_identifier,
  null as plcy_xl_risk_ind,
  null as plcy_axa_ind,
  null as trans_fin_type_fronting_ind,
  null as plcy_revenue_synergy_cd,
  null as plcy_revenue_synergy_descr,
  null as plcy_revenue_synergy_ind,
  null as plcy_eea_ind,
  null as plcy_eea_ind_dt,
  null as PLcy_ilu_ind,
  null as plcy_integrated_solution_ind
from 
  """ + conn_xl_edw_hash_xle_r32_FilePath + "." + mapVariable_TBL_DIM_N_HASH + """
where 
  src_sys_cd='""" +mapVariable_SRC_SYS_CD + """' and
  tbl_nme='""" + mapVariable_TARGET_TABLE_NAME + """' and
  isrt_updt_ind='D'
ORDER BY 
  plcy_hashnk, 
  plcy_hashfull
"""

#t_int_xle_policy = spark.sql(query)  #removed monotonically increasing id
exp_fetch_t_int_xle_policy = spark.sql(query)


In [0]:
#reformat and replace schema names with db names.  
#values in SQL query were hard-coded when they should have been variables

#Reading Data From Source
query ="""
SELECT
  t_batch.batch_key as batch_key,
  t_batch.batch_nme as batch_nme,
  t_batch.batch_sts_ind as batch_sts_ind 
FROM   
  """ + conn_xl_edw_warehouseadmin_xle_r32_FilePath + """.t_batch t_batch,
  """ + conn_xl_edw_warehouseadmin_xle_r32_FilePath + """.t_batch_set t_batch_set 
WHERE    
  t_batch.batch_set_key=t_batch_set.batch_set_key AND
  t_batch_set.batch_set_src_sys_cd='""" + mapVariable_SRC_SYS_CD + """' AND
  t_batch.batch_sts_ind='""" + mapVariable_BATCH_STS_IND + """' AND 
  t_batch.batch_nme='""" + mapVariable_WF_NAME + """' 
  and t_batch.batch_key='841813241'
"""
	
t_batch = spark.sql(query)  #removed monotonically increasing id



In [0]:
#reformat and replace schema names with db names.  
#values in SQL query were hard-coded when they should have been variables

#Reading Data From Source

query ="""
SELECT 
  PH.dim_key,
  PH.dim_id,
  NH.hashnk,
  NH.hashfull,
  NH.isrt_updt_ind,
  PH.expr_ind 
FROM
  """ + conn_xl_edw_hash_xle_r32_FilePath + "." + mapVariable_TBL_DIM_N_HASH + """ NH,
  """ + conn_xl_edw_hash_xle_r32_FilePath + "." + mapVariable_TBL_DIM_P_HASH + """ PH  
WHERE 
  NH.src_sys_cd='""" + mapVariable_SRC_SYS_CD + """' and
  PH.tbl_nme='""" + mapVariable_TARGET_TABLE_NAME + """' and
  NH.tbl_nme='""" + mapVariable_TARGET_TABLE_NAME + """' and
  PH.src_sys_cd='""" + mapVariable_SRC_SYS_CD + """' and  
  NH.hashnk=PH.hashnk and
  NH.isrt_updt_ind in('I','U') and
  PH.expr_ind='N' 

union all

SELECT 
  Max(PH.dim_key),
  PH.dim_id,
  NH.hashnk,
  NH.hashfull,
  NH.isrt_updt_ind,
  PH.expr_ind
FROM
  """ + conn_xl_edw_hash_xle_r32_FilePath + "." + mapVariable_TBL_DIM_N_HASH + """ NH,
  """ + conn_xl_edw_hash_xle_r32_FilePath + "." + mapVariable_TBL_DIM_P_HASH + """ PH 
WHERE
  NH.src_sys_cd='""" + mapVariable_SRC_SYS_CD + """' and
  PH.tbl_nme='""" + mapVariable_TARGET_TABLE_NAME + """' and
  NH.tbl_nme='""" + mapVariable_TARGET_TABLE_NAME + """' and
  PH.src_sys_cd='""" + mapVariable_SRC_SYS_CD + """' and  
  NH.hashnk=PH.hashnk and
  NH.isrt_updt_ind in('U') and
  PH.expr_ind='Y'
group by 
  PH.dim_id,
  NH.hashnk,
  NH.hashfull,
  NH.isrt_updt_ind,
  PH.expr_ind,
  NH.src_sys_cd

union all

SELECT 
  Max(PH.dim_key),
  PH.dim_id,
  NH.hashnk,
  NH.hashfull,
  NH.isrt_updt_ind,
  PH.expr_ind
FROM
  """ + conn_xl_edw_hash_xle_r32_FilePath + "." + mapVariable_TBL_DIM_N_HASH + """ NH,
  """ + conn_xl_edw_hash_xle_r32_FilePath + "." + mapVariable_TBL_DIM_P_HASH + """ PH   
WHERE 
  NH.src_sys_cd='""" + mapVariable_SRC_SYS_CD + """' and
  PH.tbl_nme='""" + mapVariable_TARGET_TABLE_NAME + """' and
  NH.tbl_nme='""" + mapVariable_TARGET_TABLE_NAME + """' and
  PH.src_sys_cd='""" + mapVariable_SRC_SYS_CD + """' and  
  NH.hashnk=PH.hashnk and
  NH.isrt_updt_ind in('D') and
  PH.expr_ind='N'
group by 
  PH.dim_id,
  NH.hashnk,
  NH.hashfull,
  NH.isrt_updt_ind,
  PH.expr_ind,
  NH.src_sys_cd

union all

SELECT 
  Max(PH.dim_key),
  PH.dim_id,
  NH.hashnk,
  NH.hashfull,
  NH.isrt_updt_ind,
  PH.expr_ind
FROM
  """ + conn_xl_edw_hash_xle_r32_FilePath + "." + mapVariable_TBL_DIM_N_HASH + """ NH,
  """ + conn_xl_edw_hash_xle_r32_FilePath + "." + mapVariable_TBL_DIM_P_HASH + """ PH   
WHERE 
  NH.src_sys_cd='""" + mapVariable_SRC_SYS_CD + """' and
  PH.tbl_nme='""" + mapVariable_TARGET_TABLE_NAME + """' and
  NH.tbl_nme='""" + mapVariable_TARGET_TABLE_NAME + """' and
  PH.src_sys_cd='""" + mapVariable_SRC_SYS_CD + """' and  
  NH.hashnk=PH.hashnk and
  NH.isrt_updt_ind in('O') and
  PH.expr_ind='N'
group by 
  PH.dim_id,
  NH.hashnk,
  NH.hashfull,
  NH.isrt_updt_ind,
  PH.expr_ind,
  NH.src_sys_cd
order by 
  3,
  4
"""

#t_previous_hashes_trn = spark.sql(query) #removed monotonically increasing id
exp_fetch_t_previous_hashes_poc = spark.sql(query)

#3.85s to here

In [0]:
#Had to modify column name.  isrt_upd_ind does not exist....it should be insrt_updt_ind
#Multi Joiner Transformation !!This was missing a column!  Also column ordering is different

jnr_intplcy_previous_hash = \
exp_fetch_t_previous_hashes_poc\
  .join(exp_fetch_t_int_xle_policy,\
    (exp_fetch_t_previous_hashes_poc.hashnk==exp_fetch_t_int_xle_policy.plcy_hashnk) & \
    (exp_fetch_t_previous_hashes_poc.hashfull==exp_fetch_t_int_xle_policy.plcy_hashfull),\
    how="inner")\
  .select(\
    exp_fetch_t_int_xle_policy["plcy_src_sys_cd"],\
    exp_fetch_t_int_xle_policy["plcy_textnk"],\
    exp_fetch_t_int_xle_policy["plcy_hashnk"],\
    exp_fetch_t_int_xle_policy["plcy_hashfull"],\
    exp_fetch_t_int_xle_policy["plcy_nbr"],\
    exp_fetch_t_int_xle_policy["plcy_eff_dt"],\
    exp_fetch_t_int_xle_policy["plcy_expr_dt"],\
    exp_fetch_t_int_xle_policy["plcy_binder_mga_ind"],\
    exp_fetch_t_int_xle_policy["plcy_ledger_ri_type_cd"],\
    exp_fetch_t_int_xle_policy["plcy_ledger_ri_type_descr"],\
    exp_fetch_t_int_xle_policy["plcy_days_to_cancel"],\
    exp_fetch_t_int_xle_policy["plcy_dir_assm_ind"],\
    exp_fetch_t_int_xle_policy["plcy_dir_assm_ind_descr"],\
    exp_fetch_t_int_xle_policy["plcy_earn_pattern_method_ind"],\
    exp_fetch_t_int_xle_policy["plcy_old_new_econ_ind"],\
    exp_fetch_t_int_xle_policy["plcy_lta_eff_dt"],\
    exp_fetch_t_int_xle_policy["plcy_lta_expr_dt"],\
    exp_fetch_t_int_xle_policy["plcy_lta_ind"],\
    exp_fetch_t_int_xle_policy["plcy_lead_follow_ind"],\
    exp_fetch_t_int_xle_policy["plcy_new_rnwl_ind"],\
    exp_fetch_t_int_xle_policy["plcy_new_rnwl_ind_descr"],\
    exp_fetch_t_int_xle_policy["plcy_primary_excess_ind"],\
    exp_fetch_t_int_xle_policy["plcy_primary_excess_ind_descr"],\
    exp_fetch_t_int_xle_policy["plcy_prog_ind"],\
    exp_fetch_t_int_xle_policy["plcy_prog_ind_descr"],\
    exp_fetch_t_int_xle_policy["plcy_uw_year"],\
    exp_fetch_t_int_xle_policy["plcy_written_dt"],\
    exp_fetch_t_int_xle_policy["plcy_share_of_whole_pct"],\
    exp_fetch_t_int_xle_policy["prior_plcy_nbr"],\
    exp_fetch_t_int_xle_policy["plcy_prog_master_ind"],\
    exp_fetch_t_int_xle_policy["plcy_prog_master_layer_ind"],\
    exp_fetch_t_int_xle_policy["plcy_orig_eff_dt"],\
    exp_fetch_t_int_xle_policy["plcy_orig_expr_dt"],\
    exp_fetch_t_previous_hashes_poc["dim_key"],\
    exp_fetch_t_previous_hashes_poc["dim_id"],\
    exp_fetch_t_previous_hashes_poc["isrt_updt_ind"],\
    exp_fetch_t_previous_hashes_poc["expr_ind"],\
    exp_fetch_t_int_xle_policy["plcy_runoff_ind"],\
    exp_fetch_t_int_xle_policy["plcy_cancelled_dt"],\
    exp_fetch_t_int_xle_policy["plcy_binder_mga_cancel_notice_pd"],\
    exp_fetch_t_int_xle_policy["plcy_market_class_cd"],\
    exp_fetch_t_int_xle_policy["plcy_market_class_descr"],\
    exp_fetch_t_int_xle_policy["plcy_mgmt_triangle_cd"],\
    exp_fetch_t_int_xle_policy["plcy_mgmt_triangle_descr"],\
    exp_fetch_t_int_xle_policy["plcy_risk_class_cd"],\
    exp_fetch_t_int_xle_policy["plcy_risk_class_descr"],\
    exp_fetch_t_int_xle_policy["plcy_syndicate_cd"],\
    exp_fetch_t_int_xle_policy["plcy_syndicate_nme"],\
    exp_fetch_t_int_xle_policy["plcy_syndicate_ssr_cd"],\
    exp_fetch_t_int_xle_policy["plcy_syndicate_ssr_descr"],\
    exp_fetch_t_int_xle_policy["plcy_title"],\
    exp_fetch_t_int_xle_policy["plcy_risk_band_cd"],\
    exp_fetch_t_int_xle_policy["plcy_risk_band_descr"],\
    exp_fetch_t_int_xle_policy["s2_plcy_status_cd"],\
    exp_fetch_t_int_xle_policy["s2_plcy_status_descr"],\
    exp_fetch_t_int_xle_policy["plcy_xl_risk_ind"],\
    exp_fetch_t_int_xle_policy["s2_plcy_status_dt"],\
    exp_fetch_t_int_xle_policy["plcy_old_new_econ_ind_descr"],\
    exp_fetch_t_int_xle_policy["plcy_upr_transition_dt"],\
    exp_fetch_t_int_xle_policy["plcy_ny_free_trd_zn_class_cd"],\
    exp_fetch_t_int_xle_policy["plcy_ny_free_trd_zn_class_descr"],\
    exp_fetch_t_int_xle_policy["plcy_captive_ind"],\
    exp_fetch_t_int_xle_policy["plcy_captive_ind_descr"],\
    exp_fetch_t_int_xle_policy["plcy_multi_year_ind"],\
    exp_fetch_t_int_xle_policy["plcy_ny_free_trd_zn_stat_cd"],\
    exp_fetch_t_int_xle_policy["plcy_ny_free_trd_zn_stat_descr"],\
    exp_fetch_t_int_xle_policy["plcy_risk_purch_grp_ind"],\
    exp_fetch_t_int_xle_policy["plcy_lgl_enty_consolidation_ind"],\
    exp_fetch_t_int_xle_policy["plcy_facilities_ind"],\
    exp_fetch_t_int_xle_policy["frame_underwriting_ref_identifier"],\
    exp_fetch_t_int_xle_policy["plcy_axa_ind"],\
    exp_fetch_t_int_xle_policy["trans_fin_type_fronting_ind"],\
    exp_fetch_t_int_xle_policy["plcy_revenue_synergy_cd"],\
    exp_fetch_t_int_xle_policy["plcy_revenue_synergy_descr"],\
    exp_fetch_t_int_xle_policy["plcy_revenue_synergy_ind"],\
    exp_fetch_t_int_xle_policy["plcy_eea_ind"],\
    exp_fetch_t_int_xle_policy["plcy_eea_ind_dt"],\
    exp_fetch_t_int_xle_policy["PLcy_ilu_ind"],\
    exp_fetch_t_int_xle_policy["plcy_integrated_solution_ind"],\
    exp_fetch_t_int_xle_policy["legacy_prior_plcy_nbr"],\
    exp_fetch_t_int_xle_policy["plcy_xl_risk_ind"]\
  )
#legacy_prior_plcy_nbr was missing from the generated code


In [0]:
#Expression Transformation - just need to append 5 columns to dataset...2 via join to lookups.  This accomplishes the same result with much less complexity than the generated code.  Since we'll be reading from this multiple times I've added .cache() to the end to increase performance
exp_pass_thruout=\
jnr_intplcy_previous_hash\
  .join(t_dw_etl_batch_process_history,t_dw_etl_batch_process_history["dummy"] == lit('X'), how="inner")\
  .join(t_batch,(t_batch["batch_nme"]==lit(mapVariable_WF_NAME)) & (t_batch["batch_sts_ind"] == lit(mapVariable_BATCH_STS_IND)), how="left")\
  .select(jnr_intplcy_previous_hash["*"],\
          t_dw_etl_batch_process_history["current_dt"],\
          t_batch["batch_key"].alias("dw_batch_id"))\
  .withColumn("dw_row_active_ind_Y", lit("Y"))\
  .withColumn("dw_row_active_ind_N", lit("N"))\
  .withColumn("dw_row_end_dt", lit(mapVariable_DEFAULT_DATE).cast("timestamp"))

exp_pass_thruout.cache()

Out[16]: DataFrame[plcy_src_sys_cd: string, plcy_textnk: string, plcy_hashnk: string, plcy_hashfull: string, plcy_nbr: string, plcy_eff_dt: date, plcy_expr_dt: date, plcy_binder_mga_ind: string, plcy_ledger_ri_type_cd: string, plcy_ledger_ri_type_descr: string, plcy_days_to_cancel: smallint, plcy_dir_assm_ind: string, plcy_dir_assm_ind_descr: string, plcy_earn_pattern_method_ind: string, plcy_old_new_econ_ind: string, plcy_lta_eff_dt: date, plcy_lta_expr_dt: date, plcy_lta_ind: string, plcy_lead_follow_ind: string, plcy_new_rnwl_ind: string, plcy_new_rnwl_ind_descr: string, plcy_primary_excess_ind: string, plcy_primary_excess_ind_descr: string, plcy_prog_ind: string, plcy_prog_ind_descr: string, plcy_uw_year: smallint, plcy_written_dt: date, plcy_share_of_whole_pct: decimal(16,8), prior_plcy_nbr: string, plcy_prog_master_ind: string, plcy_prog_master_layer_ind: string, plcy_orig_eff_dt: date, plcy_orig_expr_dt: date, dim_key: bigint, dim_id: bigint, isrt_updt_ind: string, expr_ind: string, plcy_runoff_ind: string, plcy_cancelled_dt: date, plcy_binder_mga_cancel_notice_pd: string, plcy_market_class_cd: string, plcy_market_class_descr: string, plcy_mgmt_triangle_cd: string, plcy_mgmt_triangle_descr: string, plcy_risk_class_cd: string, plcy_risk_class_descr: string, plcy_syndicate_cd: string, plcy_syndicate_nme: string, plcy_syndicate_ssr_cd: string, plcy_syndicate_ssr_descr: string, plcy_title: string, plcy_risk_band_cd: string, plcy_risk_band_descr: string, s2_plcy_status_cd: string, s2_plcy_status_descr: string, plcy_xl_risk_ind: string, s2_plcy_status_dt: date, plcy_old_new_econ_ind_descr: string, plcy_upr_transition_dt: date, plcy_ny_free_trd_zn_class_cd: string, plcy_ny_free_trd_zn_class_descr: string, plcy_captive_ind: string, plcy_captive_ind_descr: string, plcy_multi_year_ind: string, plcy_ny_free_trd_zn_stat_cd: string, plcy_ny_free_trd_zn_stat_descr: string, plcy_risk_purch_grp_ind: string, plcy_lgl_enty_consolidation_ind: string, plcy_facilities_ind: string, frame_underwriting_ref_identifier: string, plcy_axa_ind: string, trans_fin_type_fronting_ind: string, plcy_revenue_synergy_cd: string, plcy_revenue_synergy_descr: string, plcy_revenue_synergy_ind: string, plcy_eea_ind: string, plcy_eea_ind_dt: date, PLcy_ilu_ind: string, plcy_integrated_solution_ind: string, legacy_prior_plcy_nbr: string, plcy_xl_risk_ind: string, current_dt: timestamp, dw_batch_id: int, dw_row_active_ind_Y: string, dw_row_active_ind_N: string, dw_row_end_dt: timestamp]

In [0]:
#Router Transformation
#modified df names to be consistent with router group names and for clarity

rtr_insert = \
exp_pass_thruout\
  .filter((exp_pass_thruout.isrt_updt_ind == 'I' ) | ( ((exp_pass_thruout.isrt_updt_ind == 'U') &  (exp_pass_thruout.expr_ind == 'N'))))\
  .select(\
    col("dim_key").cast("integer").alias("plcy_key"),\
    col("dim_id").cast("integer").alias("plcy_id"),\
    "plcy_hashnk",\
    "plcy_hashfull",\
    "plcy_textnk",\
    "plcy_nbr",\
    "plcy_eff_dt",\
    "plcy_expr_dt",\
    "plcy_cancelled_dt",\
    "plcy_binder_mga_cancel_notice_pd",\
    "plcy_binder_mga_ind",\
    "plcy_ledger_ri_type_cd",\
    "plcy_ledger_ri_type_descr",\
    "plcy_days_to_cancel",\
    "plcy_dir_assm_ind",\
    "plcy_dir_assm_ind_descr",\
    "plcy_earn_pattern_method_ind",\
    "plcy_old_new_econ_ind",\
    "plcy_lead_follow_ind",\
    "plcy_market_class_cd",\
    "plcy_market_class_descr",\
    "plcy_mgmt_triangle_cd",\
    "plcy_mgmt_triangle_descr",\
    "plcy_lta_eff_dt",\
    "plcy_lta_expr_dt",\
    "plcy_lta_ind",\
    "plcy_new_rnwl_ind",\
    "plcy_new_rnwl_ind_descr",\
    "plcy_orig_eff_dt",\
    "plcy_orig_expr_dt",\
    "plcy_primary_excess_ind",\
    "plcy_primary_excess_ind_descr",\
    "plcy_prog_ind",\
    "plcy_prog_ind_descr",\
    "plcy_prog_master_ind",\
    "plcy_prog_master_layer_ind",\
    "plcy_risk_class_cd",\
    "plcy_risk_class_descr",\
    "plcy_runoff_ind",\
    "plcy_share_of_whole_pct",\
    "plcy_src_sys_cd",\
    "plcy_syndicate_cd",\
    "plcy_syndicate_nme",\
    "plcy_syndicate_ssr_cd",\
    "plcy_syndicate_ssr_descr",\
    "plcy_uw_year",\
    "plcy_written_dt",\
    "prior_plcy_nbr",\
    col("dw_row_active_ind_Y").alias("dw_row_active_ind"),\
    when(col("isrt_updt_ind")=='I',mapVariable_DEFAULT_START_DATE).otherwise(col("current_dt")).alias("dw_row_start_dt").cast("timestamp"),\
    "dw_row_end_dt",\
    "dw_batch_id",\
    col("current_dt").alias("dw_create_dt"),\
    col("current_dt").alias("dw_last_update_dt"),\
    "plcy_title",\
    "plcy_risk_band_cd",\
    "plcy_risk_band_descr",\
    "s2_plcy_status_cd",\
    "s2_plcy_status_descr",\
    "s2_plcy_status_dt",\
    col("plcy_upr_transition_dt").cast("timestamp"),\
    "plcy_ny_free_trd_zn_class_cd",\
    "plcy_ny_free_trd_zn_class_descr",\
    "plcy_captive_ind",\
    "plcy_lgl_enty_consolidation_ind",\
    "plcy_multi_year_ind",\
    "plcy_ny_free_trd_zn_stat_cd",\
    "plcy_ny_free_trd_zn_stat_descr",\
    "plcy_risk_purch_grp_ind",\
    "legacy_prior_plcy_nbr",\
    "plcy_facilities_ind",\
    "frame_underwriting_ref_identifier",\
    "plcy_xl_risk_ind",\
    "plcy_axa_ind",\
    "trans_fin_type_fronting_ind",\
    "plcy_revenue_synergy_cd",\
    "plcy_revenue_synergy_descr",\
    "plcy_revenue_synergy_ind",\
    "plcy_eea_ind",\
    "plcy_eea_ind_dt",\
    "PLcy_ilu_ind",\
    "plcy_integrated_solution_ind"\
  )\
  .withColumn("plcy_deleted_ind", lit("N"))\


In [0]:
#Perform inserts
rtr_insert.write.format("delta").mode('append').save(conn_Target_FilePath + conn_xl_edw_dw_xle_r32_FilePath + "/" + mapVariable_TARGET_TABLE_NAME_TEST)


In [0]:
rtr_update = \
exp_pass_thruout\
  .filter((exp_pass_thruout.isrt_updt_ind == 'U' ) & (exp_pass_thruout.expr_ind == 'Y'))\
  .select(\
    col("dim_key").cast("integer").alias("plcy_key"),\
    "dw_batch_id",\
    col("dw_row_active_ind_N").alias("dw_row_active_ind"),\
    col("current_dt").alias("dw_row_end_dt"),\
    col("current_dt").alias("dw_last_update_dt")\
  )

In [0]:
from delta.tables import *
#this should be an upsert
#exp_pass_through_tgt_upd.write.format("delta").mode('append').save(conn_xl_edw_dw_xle_r32_FilePath+"/t_d_xle_policy")

#Delta table update block
DeltaTable.forName(spark, conn_xl_edw_dw_xle_r32_FilePath + "." + mapVariable_TARGET_TABLE_NAME_TEST).alias(mapVariable_TARGET_TABLE_NAME_TEST)\
  .merge(rtr_update.alias("incoming"), mapVariable_TARGET_TABLE_NAME_TEST + ".plcy_key = incoming.plcy_key")\
  .whenMatchedUpdate(set = \
                     {"dw_batch_id" : "incoming.dw_batch_id",\
                      "dw_row_active_ind" : "incoming.dw_row_active_ind",\
                      "dw_row_end_dt" : "incoming.dw_row_end_dt",\
                      "dw_last_update_dt" : "incoming.dw_last_update_dt",\
                     })\
  .whenNotMatchedInsertAll()\
  .execute()

In [0]:
rtr_delete = \
exp_pass_thruout\
  .filter(exp_pass_thruout.isrt_updt_ind == 'D')\
  .select(\
    col("dim_key").cast("integer").alias("plcy_key"),\
    col("current_dt").alias("dw_last_update_dt"),\
    col("dw_row_active_ind_Y").alias("plcy_deleted_ind")\
  )


In [0]:
#Writing data to Target-delete
#Delta table update block
DeltaTable.forName(spark, conn_xl_edw_dw_xle_r32_FilePath + "." + mapVariable_TARGET_TABLE_NAME_TEST).alias(mapVariable_TARGET_TABLE_NAME_TEST)\
  .merge(rtr_delete.alias("incoming"), mapVariable_TARGET_TABLE_NAME_TEST + ".plcy_key = incoming.plcy_key")\
  .whenMatchedUpdate(set = \
                     {"dw_last_update_dt" : "incoming.dw_last_update_dt",
                      "plcy_deleted_ind" : "incoming.plcy_deleted_ind"\
                     })\
  .whenNotMatchedInsertAll()\
  .execute()

In [0]:
rtr_reinsert = \
exp_pass_thruout\
  .filter(exp_pass_thruout.isrt_updt_ind == 'O')\
  .select(\
    col("dim_key").cast("integer").alias("plcy_key"),\
    col("current_dt").alias("dw_last_update_dt"),\
    col("dw_row_active_ind_N").alias("plcy_deleted_ind")\
  )

In [0]:
#Writing data to Target-delete
#Delta table update block
DeltaTable.forName(spark, conn_xl_edw_dw_xle_r32_FilePath + "." + mapVariable_TARGET_TABLE_NAME_TEST).alias(mapVariable_TARGET_TABLE_NAME_TEST)\
  .merge(rtr_reinsert.alias("incoming"), mapVariable_TARGET_TABLE_NAME_TEST + ".plcy_key = incoming.plcy_key")\
  .whenMatchedUpdate(set = \
                     {"dw_last_update_dt" : "incoming.dw_last_update_dt",
                      "plcy_deleted_ind" : "incoming.plcy_deleted_ind"\
                     })\
  .whenNotMatchedInsertAll()\
  .execute()

#6.21m

In [0]:
df=spark.sql("select count(*) as processed_recs from " + conn_xl_edw_dw_xle_r32_FilePath + "." + mapVariable_TARGET_TABLE_NAME_TEST + " where dw_last_update_dt>'2021-10-01'")

In [0]:
display (df)

processed_recs
24782
